## Creacion de feature store

In [3]:
import mlflow
import feast
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [4]:
#read .csv 
data = pd.read_csv('../fase2/freshmarket_dataset.csv')
data.head()

,fecha_pedido,dia_semana,mes,dia_mes,hora_pedido,es_fin_semana,es_festivo,cliente_id,segmento_cliente,compras_previas,...,tipo_entrega,nivel_inventario_general,productos_agotados,tiempo_carga_sitio,hay_promocion,descuento_aplicado,canal_adquisicion,clima,temperatura,compra_exitosa
0,2025-10-16 04:03:12,Thursday,10,16,4,0,0,CLI_1409,Regular,17,...,Programada,99.92,4,4.86,1,5,Orgánico,Lluvioso,21.0,1
1,2024-04-14 19:51:22,Sunday,4,14,19,1,0,CLI_2424,Regular,14,...,Día siguiente,62.43,5,2.22,1,10,Referido,Nublado,29.5,1
2,2024-08-26 18:23:57,Monday,8,26,18,0,0,CLI_1434,Nuevo,0,...,Mismo día,32.52,16,3.24,1,5,Redes Sociales,Lluvioso,25.6,0
3,2025-07-12 15:16:27,Saturday,7,12,15,1,0,CLI_4611,Regular,12,...,Programada,48.18,10,3.07,0,0,Búsqueda Pagada,Soleado,25.5,1
4,2024-01-07 05:48:46,Sunday,1,7,5,1,0,CLI_7924,Regular,15,...,Mismo día,73.54,0,3.23,1,10,Búsqueda Pagada,Nublado,22.5,1


In [5]:
#summary 
data.describe(include='all')

,fecha_pedido,dia_semana,mes,dia_mes,hora_pedido,es_fin_semana,es_festivo,cliente_id,segmento_cliente,compras_previas,...,tipo_entrega,nivel_inventario_general,productos_agotados,tiempo_carga_sitio,hay_promocion,descuento_aplicado,canal_adquisicion,clima,temperatura,compra_exitosa
count,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000,10000.000000,...,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000,10000.000000,10000.000000
unique,9999,7,NaN,NaN,NaN,NaN,NaN,6047,4,NaN,...,3,NaN,NaN,NaN,NaN,NaN,5,3,NaN,NaN
top,2025-08-12 00:30:01,Thursday,NaN,NaN,NaN,NaN,NaN,CLI_7697,Regular,NaN,...,Mismo día,NaN,NaN,NaN,NaN,NaN,Orgánico,Soleado,NaN,NaN
freq,2,1503,NaN,NaN,NaN,NaN,NaN,7,4505,NaN,...,5022,NaN,NaN,NaN,NaN,NaN,3438,3836,NaN,NaN
mean,NaN,NaN,6.024400,15.693900,11.570100,0.289500,0.021600,NaN,NaN,17.049700,...,NaN,60.601456,8.074200,2.735850,0.312000,3.073500,NaN,NaN,24.920180,0.402900
std,NaN,NaN,3.241397,8.825984,6.917085,0.453553,0.145381,NaN,NaN,24.083152,...,NaN,20.527245,6.735461,1.293174,0.463333,5.334692,NaN,NaN,4.238446,0.490506
min,NaN,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,...,NaN,0.000000,0.000000,0.500000,0.000000,0.000000,NaN,NaN,18.000000,0.000000
25%,NaN,NaN,3.000000,8.000000,6.000000,0.000000,0.000000,NaN,NaN,0.000000,...,NaN,46.437500,3.000000,1.630000,0.000000,0.000000,NaN,NaN,21.400000,0.000000
50%,NaN,NaN,6.000000,16.000000,12.000000,0.000000,0.000000,NaN,NaN,8.000000,...,NaN,60.835000,6.000000,2.740000,0.000000,0.000000,NaN,NaN,24.900000,0.000000
75%,NaN,NaN,9.000000,23.000000,18.000000,1.000000,0.000000,NaN,NaN,18.000000,...,NaN,75.182500,12.000000,3.850000,1.000000,5.000000,NaN,NaN,27.400000,1.000000


In [14]:
main_features=['fecha_pedido',"cliente_id",'segmento_cliente','compras_previas',"ticket_promedio_historico","zona_entrega","compra_exitosa"]
data_filtered=data[main_features]
data_filtered["fecha_pedido"]=pd.to_datetime(data_filtered["fecha_pedido"])
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   fecha_pedido               10000 non-null  datetime64[ns]
 1   cliente_id                 10000 non-null  object        
 2   segmento_cliente           10000 non-null  object        
 3   compras_previas            10000 non-null  int64         
 4   ticket_promedio_historico  10000 non-null  float64       
 5   zona_entrega               10000 non-null  object        
 6   compra_exitosa             10000 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 547.0+ KB


C:\Users\raule\AppData\Local\Temp\ipykernel_15624\1209060180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered["fecha_pedido"]=pd.to_datetime(data_filtered["fecha_pedido"])


In [7]:
import os

os.system("feast init freshmarket_feature_store")
os.chdir("freshmarket_feature_store")

In [15]:
data_filtered.to_parquet('feature_store.parquet', index=False)

In [21]:

from feast import Entity, FeatureView, Field, FileSource,Project,ValueType,FeatureStore
from feast.types import Float64, Int64,String
from datetime import timedelta
import pandas as pd
import numpy as np
import mlflow
#Define the project
project=Project(name="freshmarket_project", description="Feature store for Fresh Market data")
# Define the entity (house)
freshmarket = Entity(
    name="cliente_id",
    value_type=ValueType.STRING,
    description="A client entity"
)
# Define the data source
file_source = FileSource(
    name="freshmarket_data_source",
    path="../feature_store.parquet",
    timestamp_field="fecha_pedido"
)


In [22]:
from feast.types import Float64, Int64,UnixTimestamp,String
from feast import Field, FeatureView
from datetime import timedelta

freshmarket_feature_view = FeatureView(
    name="freshmarket_features",
    entities=[freshmarket],
    ttl=timedelta(days=1),
    schema=[
        Field(name="segmento_cliente", dtype=String),
        Field(name="compras_previas", dtype=Int64),
        Field(name="ticket_promedio_historico", dtype=Float64),
        Field(name="zona_entrega", dtype=String),
        Field(name="compra_exitosa", dtype=Int64),
    ],
    source=file_source,
)

In [23]:
store = FeatureStore(repo_path="feature_repo")
store.apply([freshmarket, file_source, freshmarket_feature_view])

In [24]:
from datetime import datetime
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 10, 31)

store.materialize(
    start_date=START_DATE,
    end_date=END_DATE
)

Materializing 1 feature views from 2024-01-01 00:00:00+00:00 to 2025-10-31 00:00:00+00:00 into the sqlite online store.

freshmarket_features:


In [31]:
df=pd.read_parquet("feature_store.parquet")
client_ids=df["cliente_id"].tolist()
online_features=store.get_online_features(
    features=[
        "freshmarket_features:segmento_cliente",
        "freshmarket_features:compras_previas",
        "freshmarket_features:ticket_promedio_historico",
        "freshmarket_features:zona_entrega",
        "freshmarket_features:compra_exitosa",
    ],
    entity_rows=[{"cliente_id": client_id} for client_id in client_ids]
).to_df()
online_features.head()

,cliente_id,ticket_promedio_historico,compra_exitosa,compras_previas,segmento_cliente,zona_entrega
0,CLI_1409,43.30,1.0,17.0,Regular,Zona 15
1,CLI_2424,33.36,1.0,11.0,Regular,Zona 1
2,CLI_1434,28.52,0.0,12.0,Regular,Zona 15
3,CLI_4611,39.19,1.0,12.0,Regular,Villa Nueva
4,CLI_7924,0.00,1.0,0.0,Nuevo,Villa Nueva
